In [166]:
from independence_weights import *
from simulate_data import *

In [169]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
from scipy.optimize import linprog
import osqp
from scipy import sparse


def independence_weights(A, X, lambda_=0, decorrelate_moments=False, preserve_means=False, dimension_adj=True):
    n = A.shape[0]
    p = X.shape[1]
    gamma = 1
    # dif
    A = np.asarray(A).reshape(-1, 1)
    Adist = squareform(pdist(A, 'euclidean'))
    Xdist = squareform(pdist(X, 'euclidean'))

    # terms for energy-dist(Wtd A, A)
    Q_energy_A = -Adist / n ** 2
    aa_energy_A = np.sum(Adist, axis=1) / n ** 2

    # terms for energy-dist(Wtd X, X)
    Q_energy_X = -Xdist / n ** 2
    aa_energy_X = np.sum(Xdist, axis=1) / n ** 2

    mean_Adist = np.mean(Adist)
    mean_Xdist = np.mean(Xdist)

    Xmeans = np.mean(Xdist, axis=1)
    Xgrand_mean = np.mean(Xmeans)
    XA = Xdist + Xgrand_mean - np.add.outer(Xmeans, Xmeans)

    Ameans = np.mean(Adist, axis=1)
    Agrand_mean = np.mean(Ameans)
    AA = Adist + Agrand_mean - np.add.outer(Ameans, Ameans)

    # quadratic term for weighted total distance covariance
    P = XA * AA / n ** 2

    if preserve_means:
        if decorrelate_moments:
            Constr_mat = (A - np.mean(A)) * (X - np.mean(X, axis=0))
            Amat = sparse.vstack([np.eye(n), np.ones((1, n)), X.T,
                                  A.reshape(1, -1), Constr_mat.T])
            lvec = np.concatenate([np.zeros(n), [n], np.mean(
                X, axis=0), [np.mean(A)], np.zeros(X.shape[1])])
            uvec = np.concatenate(
                [np.inf * np.ones(n), [n], np.mean(X, axis=0), [np.mean(A)], np.zeros(X.shape[1])])
        else:
            Amat = sparse.vstack(
                [np.eye(n), np.ones((1, n)), X.T, A.reshape(1, -1)])
            lvec = np.concatenate(
                [np.zeros(n), [n], np.mean(X, axis=0), [np.mean(A)]])
            uvec = np.concatenate(
                [np.inf * np.ones(n), [n], np.mean(X, axis=0), [np.mean(A)]])
    else:
        if decorrelate_moments:
            Constr_mat = (A - np.mean(A)) * (X - np.mean(X, axis=0))
            Amat = sparse.vstack([np.eye(n), np.ones((1, n)), Constr_mat.T])
            lvec = np.concatenate([np.zeros(n), [n], np.zeros(X.shape[1])])
            uvec = np.concatenate(
                [np.inf * np.ones(n), [n], np.zeros(X.shape[1])])
        else:
            Amat = sparse.vstack([np.eye(n), np.ones((1, n))])
            lvec = np.concatenate([np.zeros(n), [n]])
            uvec = np.concatenate([np.inf * np.ones(n), [n]])

    if dimension_adj:
        Q_energy_A_adj = 1 / np.sqrt(p)
        Q_energy_X_adj = 1
        sum_adj = Q_energy_A_adj + Q_energy_X_adj
        Q_energy_A_adj /= sum_adj
        Q_energy_X_adj /= sum_adj
    else:
        Q_energy_A_adj = Q_energy_X_adj = 1 / 2

    for na in range(1, 50):
        p = sparse.csr_matrix(2 * (P + gamma * (Q_energy_A * Q_energy_A_adj + Q_energy_X *
                                                Q_energy_X_adj) + lambda_ * np.diag(np.ones(n)) / n ** 2))
        A = Amat

        l = lvec
        u = uvec
        q = 2 * gamma * (aa_energy_A * Q_energy_A_adj +
                         aa_energy_X * Q_energy_X_adj)
        m = osqp.OSQP()
        m.setup(P=p, q=q, A=A, l=l, u=u, max_iter=int(2e5),
                eps_abs=1e-8, eps_rel=1e-8, verbose=False)
        results = m.solve()
        if not np.any(results.x > 1e5):
            break

    weights = results.x

    weights[weights < 0] = 0

    QM_unpen = P + gamma * (Q_energy_A * Q_energy_A_adj +
                            Q_energy_X * Q_energy_X_adj)

    quadpart_unpen = weights.T @ QM_unpen @ weights
    quadpart_unweighted = np.sum(QM_unpen)

    quadpart = quadpart_unpen + np.sum(weights ** 2) * lambda_ / n ** 2

    qvec = 2 * gamma * (aa_energy_A * Q_energy_A_adj +
                        aa_energy_X * Q_energy_X_adj)
    linpart = weights @ qvec
    linpart_unweighted = np.sum(qvec)

    objective_history = quadpart + linpart + gamma * \
        (-mean_Xdist * Q_energy_X_adj - mean_Adist * Q_energy_A_adj)

    D_w = quadpart_unpen + linpart + gamma * \
        (-mean_Xdist * Q_energy_X_adj - mean_Adist * Q_energy_A_adj)
    D_unweighted = quadpart_unweighted + linpart_unweighted + gamma * \
        (-mean_Xdist * Q_energy_X_adj - mean_Adist * Q_energy_A_adj)

    qvec_full = 2 * (aa_energy_A * Q_energy_A_adj +
                     aa_energy_X * Q_energy_X_adj)

    quadpart_energy_A = weights.T @ Q_energy_A @ weights * Q_energy_A_adj
    quadpart_energy_X = weights.T @ Q_energy_X @ weights * Q_energy_X_adj
    quadpart_energy = quadpart_energy_A * \
        Q_energy_A_adj + quadpart_energy_X * Q_energy_X_adj

    distcov_history = weights.T @ P @ weights
    unweighted_dist_cov = np.sum(P)

    linpart_energy = weights @ qvec_full
    linpart_energy_A = 2 * weights @ aa_energy_A * Q_energy_A_adj
    linpart_energy_X = 2 * weights @ aa_energy_X * Q_energy_X_adj

    energy_history = quadpart_energy + linpart_energy - \
        mean_Xdist * Q_energy_X_adj - mean_Adist * Q_energy_A_adj
    energy_A = quadpart_energy_A + linpart_energy_A - mean_Adist * Q_energy_A_adj
    energy_X = quadpart_energy_X + linpart_energy_X - mean_Xdist * Q_energy_X_adj

    ess = (np.sum(weights)) ** 2 / np.sum(weights ** 2)

    ret_obj = {
        'weights': weights,
        'A': A,
        'opt': results,
        'objective': objective_history,
        'D_unweighted': D_unweighted,
        'D_w': D_w,
        'distcov_unweighted': unweighted_dist_cov,
        'distcov_weighted': distcov_history,
        'energy_A': energy_A,
        'energy_X': energy_X,
        'ess': ess
    }

    return ret_obj


def weighted_energy_stats(A, X, weights, dimension_adj=True):
    n = A.shape[0]
    p = X.shape[1]
    gamma = 1

    # Normalize weights
    weights = weights / np.mean(weights)

    A = np.asarray(A).reshape(-1, 1)
    Adist = squareform(pdist(A, 'euclidean'))
    Xdist = squareform(pdist(X, 'euclidean'))

    # Terms for energy-dist(Wtd A, A)
    Q_energy_A = -Adist / n ** 2
    aa_energy_A = np.sum(Adist, axis=1) / n ** 2

    # Terms for energy-dist(Wtd X, X)
    Q_energy_X = -Xdist / n ** 2
    aa_energy_X = np.sum(Xdist, axis=1) / n ** 2

    mean_Adist = np.mean(Adist)
    mean_Xdist = np.mean(Xdist)

    Xmeans = np.mean(Xdist, axis=1)
    Xgrand_mean = np.mean(Xmeans)
    XA = Xdist + Xgrand_mean - np.add.outer(Xmeans, Xmeans)

    Ameans = np.mean(Adist, axis=1)
    Agrand_mean = np.mean(Ameans)
    AA = Adist + Agrand_mean - np.add.outer(Ameans, Ameans)

    # Quadratic term for weighted total distance covariance
    P = XA * AA / n ** 2

    if dimension_adj:
        Q_energy_A_adj = 1 / np.sqrt(p)
        Q_energy_X_adj = 1
        sum_adj = Q_energy_A_adj + Q_energy_X_adj
        Q_energy_A_adj /= sum_adj
        Q_energy_X_adj /= sum_adj
    else:
        Q_energy_A_adj = Q_energy_X_adj = 1 / 2

    # Quadratic part of the overall objective function
    QM = P + gamma * (Q_energy_A * Q_energy_A_adj +
                      Q_energy_X * Q_energy_X_adj)
    quadpart = weights.T @ QM @ weights

    # Linear part of the overall objective function
    qvec = 2 * gamma * (aa_energy_A * Q_energy_A_adj +
                        aa_energy_X * Q_energy_X_adj)
    linpart = weights @ qvec

    # Objective function
    objective_history = quadpart + linpart + gamma * \
        (-mean_Xdist * Q_energy_X_adj - mean_Adist * Q_energy_A_adj)

    qvec_full = 2 * (aa_energy_A * Q_energy_A_adj +
                     aa_energy_X * Q_energy_X_adj)

    quadpart_energy_A = weights.T @ Q_energy_A @ weights * Q_energy_A_adj
    quadpart_energy_X = weights.T @ Q_energy_X @ weights * Q_energy_X_adj
    quadpart_energy = quadpart_energy_A * \
        Q_energy_A_adj + quadpart_energy_X * Q_energy_X_adj

    distcov_history = weights.T @ P @ weights

    linpart_energy = weights @ qvec_full
    linpart_energy_A = 2 * weights @ aa_energy_A * Q_energy_A_adj
    linpart_energy_X = 2 * weights @ aa_energy_X * Q_energy_X_adj

    # Sum of energy-dist(wtd A, A) + energy-dist(wtd X, X)
    energy_history = quadpart_energy + linpart_energy - \
        mean_Xdist * Q_energy_X_adj - mean_Adist * Q_energy_A_adj

    # Energy-dist(wtd A, A)
    energy_A = quadpart_energy_A + linpart_energy_A - mean_Adist * Q_energy_A_adj

    # Energy-dist(wtd X, X)
    energy_X = quadpart_energy_X + linpart_energy_X - mean_Xdist * Q_energy_X_adj

    ess = (np.sum(weights)) ** 2 / np.sum(weights ** 2)

    retobj = {
        'D_w': objective_history,          # The actual objective function value
        'distcov_unweighted': np.sum(P),
        'distcov_weighted': distcov_history,  # The weighted total distance covariance
        'energy_A': energy_A,              # Energy(Wtd Treatment, Treatment)
        'energy_X': energy_X,              # Energy(Wtd X, X)
        'ess': ess                         # Effective sample size
    }

    return retobj

In [170]:



# 生成模拟数据
simdat = simulate_data(seed=999, nobs=500)
# 提取响应变量、处理变量和混杂变量
y = simdat['data']['Y']
A = simdat['data']['A']
X = simdat['data'][['Z1', 'Z2', 'Z3', 'Z4', 'Z5']].values

# 计算独立权重
dcows = independence_weights(A, X)
dcows

/Users/liushucheng/miniconda3/envs/try-mps/lib/python3.8/site-packages/osqp/utils.py:122: UserWarning: Converting sparse A to a CSC (compressed sparse column) matrix. (It may take a while...)
  warn("Converting sparse A to a CSC " +


{'weights': array([2.06711194e-08, 2.44645373e+00, 0.00000000e+00, 6.11828850e-01,
        2.11775610e-01, 2.86466065e-01, 2.37657384e+00, 3.82309844e+00,
        6.89510304e+00, 2.01637015e-09, 4.69173364e-01, 7.27888793e-11,
        1.16892426e-06, 1.91988691e+00, 6.14179871e-07, 2.88950773e-01,
        2.71225521e-01, 3.36139307e+00, 2.08305404e-09, 5.30599526e-10,
        3.71570627e-01, 0.00000000e+00, 0.00000000e+00, 4.33428029e-08,
        3.93420283e-01, 5.66307800e-09, 2.40882792e+00, 4.82536608e+00,
        9.62550530e-01, 2.75208482e-01, 3.64562890e-10, 8.97143989e-08,
        1.75897526e-07, 2.47457963e-08, 4.01601425e+00, 5.36233268e-01,
        4.77015652e-09, 2.31799724e-09, 1.30017017e+00, 2.16310344e-08,
        0.00000000e+00, 6.85324200e-01, 3.18130864e-09, 1.61380850e-09,
        1.09498252e+00, 1.02381997e-09, 6.45794845e-01, 1.51052309e+01,
        5.05039041e+00, 9.53370094e-10, 2.01774379e+00, 1.55829804e-09,
        0.00000000e+00, 2.52683968e-01, 2.18461700e+0

In [162]:
import numpy as np
from scipy.sparse import csr_matrix
dense_P = np.array([[1, 0, 0],
                    [0, 2, 0],
                    [0, 0, 3]])

# 将密集矩阵转换成稀疏矩阵
sparse_P = csr_matrix(dense_P)
print(sparse_P)

  (0, 0)	1
  (1, 1)	2
  (2, 2)	3


In [163]:
n_samples = 5
n_features = 2

# 生成随机的处理变量A，混杂变量X
A = np.random.randint(0, 2, size=n_samples)  # 生成0和1之间的随机整数作为处理变量
X = np.random.randn(n_samples, n_features)   # 生成服从标准正态分布的随机混杂变量数据

# 调用独立权重函数并传入测试数据
weights_results = independence_weights(A, X)

# 打印结果
print("Weights:", weights_results['weights'])
print("Weighted dependence distance:", weights_results['D_w'])
print("Unweighted dependence distance:", weights_results['D_unweighted'])
print("Effective sample size:", weights_results['ess'])

Weights: [1.12428662 1.39597561 1.04683232 0.94697048 0.48593497]
Weighted dependence distance: 0.034053853148687696
Unweighted dependence distance: 0.060517086758700156
Effective sample size: 4.5943112064639955


In [164]:
# 生成一个规律的 5x5 矩阵作为 X
X = np.array([[1, 2, 3, 4, 5],
              [2, 3, 4, 5, 6],
              [3, 4, 5, 6, 7],
              [4, 5, 6, 7, 8],
              [5, 6, 7, 8, 9]])

# 生成一个规律的 5x1 矩阵作为 A
A = np.array([1, 2, 3, 4, 5])

# 调用 independence_weights 函数
dcows = independence_weights(A, X)
dcows

{'weights': array([0.16170485, 1.14718192, 2.38222646, 1.14718192, 0.16170485]),
 'A': <6x5 sparse matrix of type '<class 'numpy.float64'>'
 	with 10 stored elements in COOrdinate format>,
 'opt': <osqp.OSQP_results at 0x161b6a770>,
 'objective': 1.3637060753110894,
 'D_unweighted': 2.719058660639744,
 'D_w': 1.3637060753110894,
 'distcov_unweighted': 2.7190586606397438,
 'distcov_weighted': 1.0135400035675384,
 'energy_A': 0.05836101195725851,
 'energy_X': 0.2918050597862929,
 'ess': 2.9906622406812002}